[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/unboxai/examples-gallery/blob/main/tabular-classification/documentation-tutorial/tabular_tutorial_part_2.ipynb)



# Welcome to the Unbox tabular tutorial - Part 2

You should use this notebook together with the final part of the [**tabular tutorial**](https://docs.unbox.ai/docs/uploading-your-first-model-and-dataset) from our documentation. This is where we solve the identified issue affecting the first version of our model.

In [27]:
%%bash

if [ ! -e "requirements.txt" ]; then
    curl "https://raw.githubusercontent.com/unboxai/examples-gallery/main/tabular-classification/documentation-tutorial/requirements.txt" --output "requirements.txt"
fi

In [ ]:
!pip install -r requirements.txt

## 1. Loading the original training set

First, let's import the libraries we need and load the churn training set. We will then confirm the issue we have identified during the tutorial.

In [1]:
import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [2]:
TRAINING_SET_URL = "https://unbox-static-assets.s3.us-west-2.amazonaws.com/examples-datasets/tabular-classification/Churn+prediction/churn_train.csv"
VALIDATION_SET_URL = "https://unbox-static-assets.s3.us-west-2.amazonaws.com/examples-datasets/tabular-classification/Churn+prediction/churn_val.csv"

In [3]:
training_set = pd.read_csv(TRAINING_SET_URL)
val_set = pd.read_csv(VALIDATION_SET_URL)

During the tutorial, we discovered that our model was having trouble predicting samples from female users. More specifically, from retained female users, which was causing an error class to be 5x larger than the other.

We hypothesized that that was a symptom of that situation being underrepresented on the training set. This is indeed the case! 

In [4]:
training_set.groupby(["Gender", "Exited"])["Exited"].count()

Gender  Exited
Female  0           20
        1           80
Male    0         3346
        1          654
Name: Exited, dtype: int64

Notice how there are only 100 samples from female users. Out of those, only 20 are from retained female users. On the other hand, male users make up ~97% of the dataset. It is clear that we need more data for female users to remove our model's bias!

## 2. Augmenting the training set

To augment our training set, we have gotten almost 5000 new labeled samples from production.

In [5]:
NEW_PROD_DATA_URL = "https://unbox-static-assets.s3.us-west-2.amazonaws.com/examples-datasets/tabular-classification/Churn+prediction/churn_new_prod_data.csv"

In [6]:
new_prod_data = pd.read_csv(NEW_PROD_DATA_URL)
new_prod_data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,8274,15709643,Gray,675,France,Male,32,1,0.00,3,1,0,85901.09,0
1,8277,15775131,Bartlett,580,Spain,Male,32,9,142188.20,2,0,1,128028.60,0
2,8281,15745716,McGregor,706,Spain,Male,53,7,0.00,2,0,1,117939.17,0
3,8282,15598485,Pinto,567,Spain,Male,40,8,28649.64,1,1,1,95140.62,0
4,8284,15754569,Pagnotto,664,France,Male,57,1,0.00,2,1,1,56562.57,0


We are going to augment our training set with 3000 new samples from female users. Hopefully, now our training set is much more balanced!

In [7]:
female_user_data = new_prod_data[new_prod_data["Gender"] == "Female"]

In [8]:
training_set_new = pd.concat([training_set, female_user_data.iloc[:3000, :]])

## 3. Pre-process data and re-train the model

Now it's time to pre-process the data again, to encode the categorical features and re-train our gradient boosting classifier.

In [9]:
X_train = training_set_new.iloc[:, 3:-1]
y_train = training_set_new.iloc[:, -1]

X_val = val_set.iloc[:, 3:-1]
y_val = val_set.iloc[:, -1]

In [10]:
def data_encode_one_hot(df, encoders):
    """ Encodes categorical features using one-hot encoding. """
    df = df.copy(True)
    df.reset_index(drop=True, inplace=True) # Causes NaNs otherwise
    for feature, enc in encoders.items():
        print(f"encoding {feature}")
        enc_df = pd.DataFrame(enc.transform(df[[feature]]).toarray(), columns=enc.get_feature_names([feature]))
        df = df.join(enc_df)
        df = df.drop(columns=feature)
    return df

In [11]:
def create_encoder_dict(df, categorical_feature_names):
    """ Creates encoders for each of the categorical features. 
        The predict function will need these encoders. 
    """
    encoders = {}
    for feature in categorical_feature_names:
        enc = OneHotEncoder(handle_unknown='ignore')
        enc.fit(df[[feature]])
        encoders[feature] = enc
    return encoders

In [12]:
# creating the encoder dict for the categorical features (gender and geography)
encoders = create_encoder_dict(X_train, ['Geography', 'Gender'])

In [13]:
# encoding the categorical features in our training and validation sets
X_train_one_hot = data_encode_one_hot(X_train, encoders)
X_val_one_hot = data_encode_one_hot(X_val, encoders)

encoding Geography
encoding Gender
encoding Geography
encoding Gender


In [14]:
sklearn_model = GradientBoostingClassifier(random_state=42) 
sklearn_model.fit(X_train_one_hot, y_train)

GradientBoostingClassifier(random_state=42)

In [15]:
print(classification_report(y_val, sklearn_model.predict(X_val_one_hot)))

              precision    recall  f1-score   support

           0       0.90      0.95      0.92       795
           1       0.75      0.58      0.65       205

    accuracy                           0.87      1000
   macro avg       0.82      0.76      0.79      1000
weighted avg       0.87      0.87      0.87      1000



## 5. Unbox part -- have fun creating the next few cells!

Now it's up to you! We will just compute a few important quantities and functions. 

Head back to the tutorial to see how you need to fill out the next few cells.

In [16]:
feature_names = X_train.columns.values.tolist()
categorical_feature_names = ["Gender", "Geography"]
class_names = ["Retained", "Exited"]

In [17]:
def predict_proba(model, input_features: np.ndarray, col_names, one_hot_encoder, encoders):
    df = pd.DataFrame(input_features, columns=col_names)
    encoded_df = one_hot_encoder(df, encoders)
    return model.predict_proba(encoded_df.to_numpy())